# Numerical implementations of $\text{SDP}(n_1,n_2)$

In this notebook, we would like to implement the semidefinite programmings (SDP) expressed in Eq.(15) in the paper https://arxiv.org/abs/2005.08883.

In [1]:
import numpy as np
import cvxpy as cp
import math
import functools as fc
from operator import mul

import lib_non_local_games as nlg
import SDP_hierarchy as SDP

Check if Mosek is installed on the machine,

In [2]:
if not 'MOSEK' in cp.installed_solvers():
    raise RuntimeError('Please install MOSEK before running this notebook.')

As we briefly mentioned in the paper, it is better to explicitly exploit the classical structure of the optimisation variable $\rho_{(A_1Q_1T)^{n_1}(A_2Q_2\hat{T})^{n_2}(S\hat{S})}$ for real implementations as it could reduce the size of the program significantly. For simplicity, we will denote $a_1, a_2$ as $a,b$ and $q_1,q_2$ as $x,y$ in this code. Then, the optimisation variable can be expressed as

$$\rho_{(A_1Q_1T)^{n_1}(A_2Q_2\hat{T})^{n_2}(S\hat{S})} = \sum_{a^{n_1},b^{n_2},x^{n_1},y^{n_2}} \vert a^{n_1},b^{n_2},x^{n_1},y^{n_2}\rangle\langle a^{n_1},b^{n_2},x^{n_1},y^{n_2}\vert \otimes \rho_{T^{n_1}\hat{T}^{n_2}S\hat{S}}(a^{n_1},b^{n_2},x^{n_1},y^{n_2}),$$

and the SDPs in Eq.(15) can be rewritten with the classical structure as

$$ \mbox{sdp}_{n_1,n_2} (V,\pi,T) = |T|^2 \max_{\rho} \sum_{a,b,x,y} V(a,b,x,y) \, \mbox{tr}\left[\Phi_{T\hat{T}|S\hat{S}} \rho_{T\hat{T}S\hat{S}}(a,b,x,y)\right]$$
$$s.t. \quad \rho_{T^{n_1}\hat{T}^{n_2}S\hat{S}}(a^{n_1},b^{n_2},x^{n_1},y^{n_2}) \geq 0 \quad \forall a^{n_1},b^{n_2},x^{n_1},y^{n_2}\,, \quad \sum_{a^{n_1},b^{n_2},x^{n_1},y^{n_2}} \mbox{tr}\left[\rho_{T^{n_1}\hat{T}^{n_2}S\hat{S}}(a^{n_1},b^{n_2},x^{n_1},y^{n_2})\right]=1$$
$$\mbox{permutation invariance on }a^{n_1},x^{n_1}\mbox{ and the system }T^{n_1}\mbox{ with respect to the other systems}$$
$$\mbox{permutation invariance on }b^{n_2},y^{n_2}\mbox{ and the system }\hat{T}^{n_2}\mbox{ with respect to the other systems}$$
$$\sum_{a}\rho_{T^{n_1}\hat{T}^{n_2}S\hat{S}}(a^{n_1},b^{n_2},x^{n_1},y^{n_2}) = \left(\pi(x)\frac{I_T}{|T|}\right)\otimes \rho_{T^{n_1-1}\hat{T}^{n_2}S\hat{S}}(a^{n_1-1},b^{n_2},x^{n_1-1},y^{n_2})$$
$$\sum_{b}\rho_{T^{n_1}\hat{T}^{n_2}S\hat{S}}(a^{n_1},b^{n_2},x^{n_1},y^{n_2}) = \left(\pi(y)\frac{I_{\hat{T}}}{|T|}\right)\otimes \rho_{T^{n_1}\hat{T}^{n_2-1}S\hat{S}}(a^{n_1},b^{n_2-1},x^{n_1},y^{n_2-1})$$
$$\rho^{T_{(T^{n_1})}}_{T^{n_1}\hat{T}^{n_2}S\hat{S}}(a^{n_1},b^{n_2},x^{n_1},y^{n_2})\geq0\,, \quad \rho^{T_{(\hat{T}^{n_2})}}_{T^{n_1}\hat{T}^{n_2}S\hat{S}}(a^{n_1},b^{n_2},x^{n_1},y^{n_2})\geq0\,, \quad \rho^{T_{(S\hat{S})}}_{T^{n_1}\hat{T}^{n_2}S\hat{S}}(a^{n_1},b^{n_2},x^{n_1},y^{n_2})\geq0, \cdots$$

We will firstly demonstrate the implementations of the first few levels with the CHSH game, and then show the numerical examples we described in Section 2 of the paper: the rule matrix $W$ and $I_{3322}$.

# CHSH game

Parameters of the game

In [3]:
# Cardinality of Alice and Bob's set of answers
answers = (2,2)

# Cardinality of Alice and Bob's set of questions
questions = (2,2)

# Dimension of the assisting quantum system
dim_assistance = 2

# Rule function of the game
rule_function = nlg.CHSH_rule_function_A1Q1A2Q2

# Probability distribution of the game
probQ1 = (1/2,1/2)
probQ2 = (1/2,1/2)
distributions = [probQ1,probQ2]

## Linear programming ($|T|=1$)

In this case, Alice and Bob do not have access to an assisting quantum system, and the problem boils down to the classical one. 

Each block $\rho_{T^{n_1}\hat{T}^{n_2}S\hat{S}}(a^{n_1},b^{n_2},x^{n_1},y^{n_2})$ in the optimisation variable becomes a number depending on $a^{n_1},b^{n_2},x^{n_1},y^{n_2}$. This means that the whole variable becomes an extendible classical probability distribution $p(a^{n_1},b^{n_2},x^{n_1},y^{n_2})$, and the SDPs become linear programmings (LP).

Since it is enough to extend only one part in the classical case, we can simplify the SDP hierarchy for $|T|=1$ as
$$\mbox{lp}_n (V,\pi,1) = \max_{p} \sum_{a,b,x,y} V(a,b,x,y) \, p(a,b,x,y)$$
$$s.t. \quad p(a,b^n,x,y^n) \geq 0 \quad \forall a,b^n,x,y^n\,, \quad \sum_{a,b^n,x,y^n} p(a,b^n,x,y^n)=1$$
$$\mbox{permutation invariance on }b^{n},y^{n}\mbox{ with respect to } a,x$$
$$\sum_{a}p(a,b^n,x,y^n) = \pi(x)\,p(b^n,y^n)$$
$$\sum_{b}p(a,b^n,x,y^n) = \pi(y)\,p(a,b^{n-1},x,y^{n-1})$$

In [7]:
# Cardinality of answer/question sets for Alice
subs_A1Q1 = (answers[0], questions[0])

# Cardinality of answer/question sets for Bob
subs_A2Q2 = (answers[1], questions[0])

# This function solves the LP for the CHSH game with no quantum assistance
#
def CHSH_classical(rule_function,subs_A1Q1,subs_A2Q2,probQ1,probQ2,level=1):
    
    ## CLASSICAL CONSTRAINTS
    constraints = []
    classical_prob, BtI_ext = nlg.classical_constraints(constraints,subs_A1Q1,subs_A2Q2,probQ1,probQ2,level)
    
    ## OBJECTIVE FUNCTION
    indices_A1Q1A2Q2 = nlg.indices_list(subs_A1Q1+subs_A2Q2)
    indices_A2Q2_ext_but_one = nlg.indices_list(subs_A2Q2*(level-1))
    
    object_function = 0
    
    # The object function is
    for index_A1Q1A2Q2 in indices_A1Q1A2Q2:
        indices_a1q1a2q2_ext = nlg.fuse_arrays(np.array([index_A1Q1A2Q2]),indices_A2Q2_ext_but_one)
        object_function += rule_function(*index_A1Q1A2Q2) * sum([classical_prob[BtI_ext(i)] for i in indices_a1q1a2q2_ext])
    
    ## PROBLEM
    prob = cp.Problem(cp.Maximize(object_function), constraints)
    result = prob.solve(solver='MOSEK')
    
    return result

The first level (= the non-signalling value)

In [6]:
CHSH_classical(rule_function,subs_A1Q1,subs_A2Q2,probQ1,probQ2)

1.0000000010619423

The second level

In [10]:
CHSH_classical(rule_function,subs_A1Q1,subs_A2Q2,probQ1,probQ2,2)

0.7500000000123522

The third level

In [11]:
CHSH_classical(rule_function,subs_A1Q1,subs_A2Q2,probQ1,probQ2,3)

0.7500000000056621

The forth level

In [12]:
CHSH_classical(rule_function,subs_A1Q1,subs_A2Q2,probQ1,probQ2,4)

0.75000000130713

## SDP(1,1)

$\mbox{sdp}_{1,1}$ can be explicitly written as
$$ \mbox{sdp}_{1,1} (V,\pi,T) = |T|^2 \max_{\rho} \sum_{a,b,x,y} V(a,b,x,y) \, \mbox{tr}\left[\Phi_{T\hat{T}|S\hat{S}} \rho_{T\hat{T}S\hat{S}}(a,b,x,y)\right]$$
$$\text{such that} \quad \rho_{T\hat{T}S\hat{S}}(a,b,x,y) \geq 0 \quad \forall a,b,x,y\,, \quad \sum_{a,b,x,y} \mbox{tr}\left[\rho_{T\hat{T}S\hat{S}}(a,b,x,y)\right]=1$$
$$\sum_{a}\rho_{T\hat{T}S\hat{S}}(a,b,x,y) = \left(\pi(x)\frac{I_{T}}{|T|}\right)\otimes \rho_{\hat{T}S\hat{S}}(b,y)$$
$$\sum_{b}\rho_{T\hat{T}S\hat{S}}(a,b,x,y) = \left(\pi(y)\frac{I_{\hat{T}}}{|T|}\right)\otimes \rho_{TS\hat{S}}(a,x)$$
$$\rho^{T_{T}}_{T\hat{T}S\hat{S}}(a,b,x,y)\geq0\,, \quad \rho^{T_{\hat{T}}}_{T\hat{T}S\hat{S}}(a,b,x,y)\geq0\,, \quad \rho^{T_{S\hat{S}}}_{T\hat{T}S\hat{S}}(a,b,x,y)\geq0.$$

### The Program

In [24]:
# SDP level (n1,n2)
level = (1,1)

# Create the SDP relaxation
CHSH = SDP.SDP_relaxation(level, answers, questions, dim_assistance, rule_function, distributions)

# Create the variable of the problem
CHSH.initialize_variable()

# Create the objective function based on the rule function passed
CHSH.create_objective_function()

# Create the constraints
CHSH.state_constraint()
CHSH.full_permutation_constraints()
CHSH.linear_constraint_Alice()
CHSH.linear_constraint_Bob()
CHSH.PPT_constraints()
CHSH.NPA1_constraint()

# Create the problem and solve it
prob = CHSH.write_problem()

### Solutions

SDP(1,1)

In [16]:
SDP11 = prob.solve(solver='MOSEK',verbose=False)
print(SDP11)

1.0000000001446023


SDP(1,1) + PPT

In [14]:
SDP11_PPT = prob.solve(solver='MOSEK',verbose=False)
print(SDP11_PPT)

1.0000000001136375


SDP(1,1) + PPT + NPA1 (with the projective assumption)

In [21]:
SDP11_PPT_NPA1proj = prob.solve(solver='MOSEK',verbose=False)
print(SDP11_PPT_NPA1proj)

0.8535533905976226


SDP(1,1) + PPT + NPA1 (without the projective assumption)

In [25]:
SDP11_PPT_NPA1 = prob.solve(solver='MOSEK',verbose=False)
print(SDP11_PPT_NPA1)

0.8535533905972075


SDP(1,1) + PPT + The projective measurement assumption

In [27]:
SDP11_PPT_proj = prob11.solve(verbose=False,solver='MOSEK')
print(SDP11_PPT_proj)

0.8535533907161744


## SDP(2,1)

$\mbox{sdp}_{2,1}$ can be explicitly written as
$$ \mbox{sdp}_{2,1} (V,\pi,T) = |T|^2 \max_{\rho} \sum_{a,b,x,y} V(a,b,x,y) \, \mbox{tr}\left[\Phi_{T\hat{T}|S\hat{S}} \rho_{T\hat{T}S\hat{S}}(a,b,x,y)\right]$$
$$\text{such that} \quad \rho_{T_1T_2\hat{T}S\hat{S}}(a_1,a_2,b,x_1,x_2,y) \geq 0 \quad \forall a_1,a_2,b,x_1,x_2,y\,, \quad \sum_{a_1,a_2,b,x_1,x_2,y} \mbox{tr}\left[\rho_{T_1T_2\hat{T}S\hat{S}}(a_1,a_2,b,x_1,x_2,y)\right]=1$$
$$\left(F_{T_1|T_2}\otimes I_{\hat{T}S\hat{S}}\right) \rho_{T_1T_2\hat{T}S\hat{S}}(a_2,a_1,b,x_2,x_1,y) \left(F_{T_1|T_2}\otimes I_{\hat{T}S\hat{S}}\right) = \rho_{T_1T_2\hat{T}S\hat{S}}(a_1,a_2,b,x_1,x_2,y)$$
$$\sum_{a_1}\rho_{T_1T_2\hat{T}S\hat{S}}(a_1,a_2,b,x_1,x_2,y) = \left(\pi(x_1)\frac{I_{T_1}}{|T|}\right)\otimes \rho_{T_2\hat{T}S\hat{S}}(a_2,b,x_2,y)$$
$$\sum_{b}\rho_{T_1T_2\hat{T}S\hat{S}}(a_1,a_2,b,x_1,x_2,y) = \left(\pi(y)\frac{I_{\hat{T}}}{|T|}\right)\otimes \rho_{T_1T_2S\hat{S}}(a_1,a_2,x_1,x_2)$$
$$\rho^{T_{T_1T_2}}_{T_1T_2\hat{T}S\hat{S}}(a_1,a_2,b,x_1,x_2,y)\geq0\,, \quad \rho^{T_{\hat{T}}}_{T_1T_2\hat{T}S\hat{S}}(a_1,a_2,b,x_1,x_2,y)\geq0\,, \quad \rho^{T_{S\hat{S}}}_{T_1T_2\hat{T}S\hat{S}}(a_1,a_2,b,x_1,x_2,y)\geq0\,, \quad \ldots$$

### The Program

In [34]:
# SDP level (n1,n2)
level = (2,1)

# Create the SDP relaxation
CHSH = SDP.SDP_relaxation(level, answers, questions, dim_assistance, rule_function, distributions)

# Create the variable of the problem
CHSH.initialize_variable()

# Create the objective function based on the rule function passed
CHSH.create_objective_function()

# Create the constraints
CHSH.state_constraint()
CHSH.full_permutation_constraints()
CHSH.linear_constraint_Alice()
CHSH.linear_constraint_Bob()
CHSH.PPT_constraints()
CHSH.NPA1_constraint()

# Create the problem and solve it
prob = CHSH.write_problem()

### Solutions

SDP(2,1)

In [29]:
SDP21 = prob.solve(verbose=False, solver='MOSEK')
print(SDP21)

0.8943377920088351


SDP(2,1) + PPT

In [31]:
SDP21_PPT = prob.solve(verbose=False, solver='MOSEK')
print(SDP21_PPT)

0.8750000000397471


SDP(2,1) + PPT + NPA1 (with the projective assumption)

In [33]:
SDP21_PPT_NPA1proj = prob.solve(verbose=False, solver='MOSEK')
print(SDP21_PPT_NPA1proj)

0.853553390752552


SDP(2,1) + PPT + NPA1 (without the projective assumption)

In [35]:
SDP21_PPT_NPA1 = prob.solve(verbose=False, solver='MOSEK')
print(SDP21_PPT_NPA1)

0.8535533905983419


## SDP(2,2)

$\mbox{sdp}_{2,2}$ can be explicitly written as
$$ \mbox{sdp}_{2,2} (V,\pi,T) = |T|^2 \max_{\rho} \sum_{a,b,x,y} V(a,b,x,y) \, \mbox{tr}\left[\Phi_{T\hat{T}|S\hat{S}} \rho_{T\hat{T}S\hat{S}}(a,b,x,y)\right]$$
$$\text{such that} \quad \rho_{T_1T_2\hat{T}_1\hat{T}_2S\hat{S}}(a_1,a_2,b_1,b_2,x_1,x_2,y_1,y_2) \geq 0 \quad \forall a_1,a_2,b_1,b_2,x_1,x_2,y_1,y_2$$ $$\sum_{a_1,a_2,b_1,b_2,x_1,x_2,y_1,y_2} \mbox{tr}\left[\rho_{T_1T_2\hat{T}_1\hat{T}_2S\hat{S}}(a_1,a_2,b_1,b_2,x_1,x_2,y_1,y_2)\right]=1$$
$$\left(F_{T_1|T_2}\otimes I_{\hat{T}_1\hat{T}_2S\hat{S}}\right) \rho_{T_1T_2\hat{T}_1\hat{T}_2S\hat{S}}(a_2,a_1,b_1,b_2,x_2,x_1,y_1,y_2) \left(F_{T_1|T_2}\otimes I_{\hat{T}_1\hat{T}_2S\hat{S}}\right) = \rho_{T_1T_2\hat{T}_1\hat{T}_2S\hat{S}}(a_1,a_2,b_1,b_2,x_1,x_2,y_1,y_2)$$
$$\left(F_{\hat{T}_1|\hat{T}_2}\otimes I_{T_1T_2S\hat{S}}\right) \rho_{T_1T_2\hat{T}_1\hat{T}_2S\hat{S}}(a_1,a_2,b_2,b_1,x_1,x_2,y_2,y_1) \left(F_{\hat{T}_1|\hat{T}_2}\otimes I_{T_1T_2S\hat{S}}\right) = \rho_{T_1T_2\hat{T}_1\hat{T}_2S\hat{S}}(a_1,a_2,b_1,b_2,x_1,x_2,y_1,y_2)$$
$$\sum_{a_1}\rho_{T_1T_2\hat{T}_1\hat{T}_2S\hat{S}}(a_1,a_2,b_1,b_2,x_1,x_2,y_1,y_2) = \left(\pi(x_1)\frac{I_{T_1}}{|T|}\right)\otimes \rho_{T_2\hat{T}_1\hat{T}_2S\hat{S}}(a_2,b_1,b_2,x_2,y_1,y_2)$$
$$\sum_{b_1}\rho_{T_1T_2\hat{T}_1\hat{T}_2S\hat{S}}(a_1,a_2,b_1,b_2,x_1,x_2,y_1,y_2) = \left(\pi(y_1)\frac{I_{\hat{T}_1}}{|T|}\right)\otimes \rho_{T_1T_2\hat{T}_2S\hat{S}}(a_1,a_2,b_2,x_1,x_2,y_2)$$
$$\rho^{T_{T_1T_2}}_{T_1T_2\hat{T}_1\hat{T}_2S\hat{S}}(a_1,a_2,b_1,b_2,x_1,x_2,y_1,y_2)\geq0\,, \quad \rho^{T_{\hat{T}_1\hat{T}_2}}_{T_1T_2\hat{T}_1\hat{T}_2S\hat{S}}(a_1,a_2,b_1,b_2,x_1,x_2,y_1,y_2)\geq0\, \quad \rho^{T_{S\hat{S}}}_{T_1T_2\hat{T}_1\hat{T}_2S\hat{S}}(a_1,a_2,b_1,b_2,x_1,x_2,y_1,y_2)\geq0\,, \quad \ldots$$

### The Program

In [29]:
# SDP level (n1,n2)
level = (2,2)

# Create the SDP relaxation
CHSH = SDP.SDP_relaxation(level, answers, questions, dim_assistance, rule_function, distributions)

# Create the variable of the problem
CHSH.initialize_variable()

# Create the objective function based on the rule function passed
CHSH.create_objective_function()

# Create the constraints
CHSH.state_constraint()
CHSH.full_permutation_constraints()
CHSH.linear_constraint_Alice()
CHSH.linear_constraint_Bob()
CHSH.PPT_constraints()
CHSH.NPA1_constraint()

# Create the problem and solve it
prob = CHSH.write_problem()

### Solutions

SDP(2,2)

In [ ]:
SDP22 = prob22.solve(verbose=False, solver='MOSEK')

SDP22 = 0.8792120993330009

The top memory requirement was 229Gb. I ran it with the computing department computer (500Gb RAM).

=================================================================

From here, even a 500gb RAM computer is not enough.

In [ ]:
SDP22_PPT = prob22.solve(verbose=False, solver='MOSEK')
print(SDP22)

## SDP(3,1)

$\mbox{sdp}_{3,1}$ can be written explicitly as
$$ \mbox{sdp}_{3,1} (V,\pi,T) = |T|^2 \max_{\rho} \sum_{a,b,x,y} V(a,b,x,y) \, \mbox{tr}\left[\Phi_{T\hat{T}|S\hat{S}} \rho_{T\hat{T}S\hat{S}}(a,b,x,y)\right]$$
$$\text{such that} \quad \rho_{T_1T_2T_3\hat{T}S\hat{S}}(a_1,a_2,a_3,b,x_1,x_2,x_3,y) \geq 0 \quad \forall a_1,a_2,a_3,b,x_1,x_2,x_3,y$$ $$ \sum_{a_1,a_2,a_3,b,x_1,x_2,x_3,y} \mbox{tr}\left[\rho_{T_1T_2T_3\hat{T}S\hat{S}}(a_1,a_2,a_3,b,x_1,x_2,x_3,y)\right]=1$$
$$\left(F_{T_1|T_2}\otimes I_{T_3\hat{T}S\hat{S}}\right) \rho_{T_1T_2T_3\hat{T}S\hat{S}}(a_2,a_1,a_3,b,x_2,x_1,x_3,y) \left(F_{T_1|T_2}\otimes I_{T_3\hat{T}S\hat{S}}\right) = \rho_{T_1T_2T_3\hat{T}S\hat{S}}(a_1,a_2,a_3,b,x_1,x_2,x_3,y)$$
$$\left(F_{T_2|T_3}\otimes I_{T_1\hat{T}S\hat{S}}\right) \rho_{T_1T_2T_3\hat{T}S\hat{S}}(a_1,a_3,a_2,b,x_1,x_3,x_2,y) \left(F_{T_2|T_3}\otimes I_{T_1\hat{T}S\hat{S}}\right) = \rho_{T_1T_2T_3\hat{T}S\hat{S}}(a_1,a_2,a_3,b,x_1,x_2,x_3,y)$$
$$\sum_{a_1}\rho_{T_1T_2T_3\hat{T}S\hat{S}}(a_1,a_2,a_3,b,x_1,x_2,x_3,y) = \left(\pi(x_1)\frac{I_{T_1}}{|T|}\right)\otimes \rho_{T_2T_3\hat{T}S\hat{S}}(a_2,a_3,b,x_2,x_3,y)$$
$$\sum_{b}\rho_{T_1T_2T_3\hat{T}S\hat{S}}(a_1,a_2,a_3,b,x_1,x_2,x_3,y) = \left(\pi(y)\frac{I_{\hat{T}}}{|T|}\right)\otimes \rho_{T_1T_2T_3S\hat{S}}(a_1,a_2,a_3,x_1,x_2,x_3)$$
$$\rho^{T_{T_1T_2T_3}}_{T_1T_2T_3\hat{T}S\hat{S}}(a_1,a_2,a_3,b,x_1,x_2,x_3,y)\geq0\,, \quad \rho^{T_{\hat{T}}}_{T_1T_2T_3\hat{T}S\hat{S}}(a_1,a_2,a_3,b,x_1,x_2,x_3,y)\geq0\,, \quad \rho^{T_{S\hat{S}}}_{T_1T_2T_3\hat{T}S\hat{S}}(a_1,a_2,a_3,b,x_1,x_2,x_3,y)\geq0\,, \quad \ldots$$

### The Program

In [ ]:
# SDP level (n1,n2)
level = (3,1)

# Create the SDP relaxation
CHSH = SDP.SDP_relaxation(level, answers, questions, dim_assistance, rule_function, distributions)

# Create the variable of the problem
CHSH.initialize_variable()

# Create the objective function based on the rule function passed
CHSH.create_objective_function()

# Create the constraints
CHSH.state_constraint()
CHSH.full_permutation_constraints()
CHSH.linear_constraint_Alice()
CHSH.linear_constraint_Bob()
CHSH.PPT_constraints()
CHSH.NPA1_constraint()

# Create the problem and solve it
prob = CHSH.write_problem()

### Solutions

SDP(3,1)

In [ ]:
SDP31 = prob31.solve(verbose=False, solver='MOSEK')
print(SDP31)

SDP31 = 0.8717348970635381

The highest memory requirement was 300Gb. I could run it with the computing department computer (500Gb RAM).

=================================================================

From here, even a 500gb RAM computer is not enough.

In [ ]:
SDP31_PPT = prob31.solve(verbose=False, solver='MOSEK')
print(SDP31_PPT)

# $I_{3322}$ game

### Define the dimensions of the game

In [36]:
dimA1 = 2
dimA2 = 2
dimQ1 = 3
dimQ2 = 3

dimT = 2

### Inputs of the program

- $V(a_1,a_2,q_1,q_2)$: I3322 rule function
- $\pi(q1,q2)$: the uniform distribution

In [22]:
probQ1 = (1/3,1/3,1/3)
probQ2 = (1/3,1/3,1/3)

rule_function = lambda a,x,b,y : nlg.general_I3322_rule_ineq(a,x,b,y,probQ1,probQ2)

## $\text{SDP}^{\text{proj}}(3,3)$ 

We make use of the results in Appendix F of https://arxiv.org/abs/2005.08883 to simplify our SDP. This simplified form holds when Alice and Bob are restricted to rank-1 projective measurement.

### Variables and Functions

In [23]:
# Subsystems A1 Q1 A2 Q2
subs_A1Q1A2Q2 = (dimA1,dimQ1,dimA2,dimQ2)
indices_A1Q1A2Q2 = nlg.indices_list(subs_A1Q1A2Q2)
dim_A1Q1A2Q2 = fc.reduce(mul, subs_A1Q1A2Q2, 1)

# Subsystems T_q1=1 ... T_q1=|Q1| T_q2=1 ... T_q2=|Q2|
subs_W = tuple(np.full(dimQ1+dimQ2, dimT))
dim_W = fc.reduce(mul, subs_W, 1)

### Optimisation

In [39]:
## VARIABLE
W = cp.Variable((dim_W,dim_W),symmetric=True)

## OBJECTIVE FUNCTION

# Build the operator connected to the inequality I
I_operator = nlg.I_operator(rule_function,indices_A1Q1A2Q2,dimQ1,dimQ2,dimT)

# Objective function
objective_function = cp.trace(cp.matmul(W,I_operator))
    
## CONSTRAINTS
constraints = []
    
# 1) rho_TTSS are (sub-normalized) quantum states
# 1a) trace of the sum is 1
constraints.append( cp.trace(W) - 1 == 0 )

# 1b) positive semidefinite matrices
constraints.append( W >> 0 )

# 2) PPT
PPT_dim = (2,)*(dimQ1+dimQ2-1)
PPT_list = [np.concatenate((np.full(2,item[0]),item[1:])) for item in nlg.indices_list(PPT_dim)]

for PPT in PPT_list:
    
    if (sum(PPT) == 0) or (sum(PPT) == 6):
        continue
    
    constraints.append( nlg.partial_transpose(W,subs_W,tuple(PPT)) >> 0 )

# Write the problem
prob = cp.Problem(cp.Maximize(objective_function), constraints)

SDP(3,3) with projective assumption (only PPT constraints are imposed). This runs on a laptop.

In [25]:
prob.solve(verbose=True, solver='MOSEK')



Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 126977          
  Cones                  : 0               
  Scalar variables       : 2080            
  Matrix variables       : 31              
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 126977          
  Cones                  : 0               
  Scalar variables       : 2080            
  Matrix variables       : 31              
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 126977
Optimizer  - Cones                  : 1
Optimizer  - Scalar variab

0.25000003971793644

# The rule matrix W in Table 1 (arXiv:2005.08883)

This is one example of our SDPs beating the first level NPA values. We could beat the first level NPA value for the rule matrix W [Table 1, arXiv:2005.08883] with SDP(2,1) + PPT + NPA1(without the projective assumption).

### Define the dimensions of the game

In [ ]:
dimA1 = 3
dimA2 = 3
dimQ1 = 2
dimQ2 = 2

dimT = 2
dimS = 2

### Inputs of the program

- $V(a_1,a_2,q_1,q_2)$: The rule matrix W in Table 1 in 'arXiv:2005.08883'
- $\pi(q1,q2)$: the uniform distribution

In [ ]:
probQ1 = (1/2,1/2)
probQ2 = (1/2,1/2)

# Rule matrix a1,q1,a2,q2 -> Rule_matrix[StI([a1,q1,a2,q2])]
Rule_matrix = [0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0]

## SDP(2,1)

### Variables and Functions

In [ ]:
# Subsystems A1_1 A1_2 Q1_1 Q1_2 A2 Q2
subs_AAQQ1_AQ2 = (dimA1,dimA1,dimQ1,dimQ1,dimA2,dimQ2)
indices_AAQQ1_AQ2 = nlg.indices_list(subs_AAQQ1_AQ2)

# Subsystems T1_1 T1_2 T2 S1 S2
subs_TT1_T2_S1_S2 = (dimT,dimT,dimT,dimS,dimS)
dim_TT1_T2_S1_S2 = fc.reduce(mul, subs_TT1_T2_S1_S2, 1)

# Subsystems A1_1 Q1_1 A2 Q2
subs_A1Q1A2Q2 = (dimA1,dimQ1,dimA2,dimQ2)
indices_A1Q1A2Q2 = nlg.indices_list(subs_A1Q1A2Q2)

# Subsystems A1 Q1
subs_A1Q1 = (dimA1,dimQ1)
indices_A1Q1 = nlg.indices_list(subs_A1Q1)

# Subsystems A2 Q2
subs_A2Q2 = (dimA2,dimQ2)
indices_A2Q2 = nlg.indices_list(subs_A2Q2)

# Maximally entangled vectors between T|S and TT|SS
phi_TS = nlg.bipartite_unnorm_max_entangled_state(dimT)
phi_TSTS = nlg.tensor([phi_TS,phi_TS])

# Maximally entangled states between TT|SS (correct order of subsystems)
Phi_TSTS = np.outer(phi_TSTS,phi_TSTS)
P1 = nlg.permutation_matrix((0,1,2,3), (0,2,1,3), (dimT, dimS, dimT, dimS))
Phi_TTSS = P1 @ Phi_TSTS @ P1

# A function which converts [a1_1,a1_2,q1_1,q1_2,a2,q2] to the corresponding index
StI = lambda seq : nlg.seqtoint(seq, subs_AAQQ1_AQ2)
StI4 = lambda seq : nlg.seqtoint(seq, subs_A1Q1A2Q2)

### Optimisation

In [ ]:
## VARIABLES 
# The (sub-normalized) states we optimize over
rho_T1T2TSS = []
for i in map(StI,indices_AAQQ1_AQ2):
    rho_T1T2TSS.append( cp.Variable((dim_TT1_T2_S1_S2,dim_TT1_T2_S1_S2),symmetric=True) )

    
## OBJECTIVE FUNCTION
# The reduced variable on TTSS and A1Q1A2Q2
rho_TTSS = lambda a1,q1,a2,q2 : nlg.rho_reduced(rho_T1T2TSS,a1,q1,a2,q2,2,1,dimT,dimS,StI,indices_A1Q1)

# The object function is
object_function = (dimT**2)*sum([int(Rule_matrix[StI4(index)])*cp.trace(Phi_TTSS@rho_TTSS(*index)) for index in indices_A1Q1A2Q2])


## CONSTRAINTS
constraints = []

# 1) rho_T1T2TSS are (sub-normalized) quantum states
# 1a) trace of the sum is 1
constraints.append( sum([cp.trace(rho_T1T2TSS[i]) for i in map(StI,indices_AAQQ1_AQ2)]) - 1 == 0 )

# 1b) positive semidefinite matrices
for i in map(StI,indices_AAQQ1_AQ2):
    constraints.append( rho_T1T2TSS[i] >> 0 )
    
# 2) Permutation invariance
nlg.full_permutation_constraints(rho_T1T2TSS,constraints,2,1,subs_TT1_T2_S1_S2,StI,indices_AAQQ1_AQ2)

# 3) First linear constraint
nlg.linear_constraint_Alice(rho_T1T2TSS,probQ1,constraints,2,1,subs_A1Q1,subs_A2Q2,dimT,dimS,StI)
    
# 4) Second linear constraint
nlg.linear_constraint_Bob(rho_T1T2TSS,probQ2,constraints,2,1,subs_A1Q1,subs_A2Q2,dimT,dimS,StI)
    
# 5) PPT criterium
for i in map(StI,indices_AAQQ1_AQ2):
    nlg.PPT_constraints(rho_T1T2TSS[i],constraints,2,1,subs_TT1_T2_S1_S2)

# 6) NPA style constraint (see PhysRevLett.98.010401)
nlg.NPA1_constraint(rho_T1T2TSS,constraints,2,1,subs_A1Q1,subs_A2Q2,dimT,dimS,probQ1,probQ2,StI,0)
# As |A|=3 and the system is qubits, we cannot assume the projective measurement anymore.


## PROBLEM
# Write the problem
probW = cp.Problem(cp.Maximize(object_function), constraints)

In [ ]:
SDPW_PPT_NPA1 = probW.solve(verbose=False, solver='MOSEK')
print(SDPW_PPT_NPA1)

0.7988756952145639

This is strictly smaller than the first level NPA value 0.8015682753138356, i.e. $\mbox{sdp}^{\mbox{NPA1}}_{2,1}(W,\pi,2) = 0.7988756952145639 < \mbox{NPA1}(W,\pi)=0.8015682753138356$